# 第十二章 目标检测  
## 12.1 简介  
- 目标检测的目的是识别图像中每个目标的类别并且对它们进行定位。  
- **定位(location)**即确定目标的位置和尺寸，通常需要勾勒出目标轮廓的**包围盒(bounding box)**，确定该包围盒的中心点坐标及宽和高。通常会有n个包围盒。  
## 12.2 度量标准mAP  
- 最常用的目标检测评价指标是**全类平均精度(mean Average Precision, mAP)**  
- mAP的计算通常是以下流程：  
  (1)IoU, 对于图像中的一个目标，计算其真实框和预测框的IoU重合度，若IoU大于给定的阈值，则认定该预测框命中了目标。
  (2)TP(True Positive), 真阳性，表示预测框正确命中了目标，即"模型预测为正样本且实际上就是正样本"的数量。比如该框与目标框有一定的重合度且达到阈值标准。  
  (3)FP(False Positive), 假阳性，表示预测框没有命中目标，未达到阈值，即"模型预测为正样本但实际上是负样本"的数量。比如该框与目标框有一定的重合度但是未能达到阈值标准。  
  (4)FN(False Negative), 假阴性，表示没有为该目标框预测出预测框，即没有察觉到这个目标框，即"模型预测为负样本但实际上是正样本"，当没有预测框时，就没有 IoU 可计算，而是直接将该真实框视为一个 漏检（False Negative）。
  (5)Precision, 查准率，即正确检测相对于所有预测包围盒的比例，  
  $$
    precision = \frac{TP}{TP + FP}  
  $$
  (6)Recall, 查全率，即正确预测相对于所有真实目标框的比例，  
  $$
    recall = \frac{TP}{TP + FN}
  $$
  (6)PR, pricision-recall曲线，由于IoU阈值间接影响precesion以及recall，因此，通过调整IoU阈值的大小，会得到不同的查全率跟查准率，因此可以做出一条PR曲线，则围出来的面积定义为AP，mAP是各个类别AP的平均值。

## 12.3 目标检测模型  
- 目标检测通常是基于**滑动窗口**，用窗口在图像上滑动，滑动到一个位置，就通过一个分类器判断该位置窗口是否存在目标。这样的弊端是尝试次数过多，时间太慢。  
- 改进方法**区域卷积神经网络(Region convolutional neural network, R-CNN)**：基于**候选目标检测**找到可能存在目标的候选窗口，而这些候选窗口的数量远远小于所有滑动窗口的数量，这是R-CNN改善检测速度的基本思想。  
- 候选目标检测指的是从图像中生成一组可能包含物体的区域（候选框 / proposals），供后续的分类器进行进一步判断和精细分类。   

## 12.4 R-CNN(2014年)  
- R-CNN大体步骤如下：
  (1)从输入图像中提取1000~2000个候选区域(候选目标检测方法常用的是Selective Search，用于高效生成可能包含物体的区域)  
  (2)利用深度卷积神经网络(CNN)提取每个候选区域的特征(CNN常用AlexNet、VGG、ResNet等)  
  (3)基于提取的深度卷积特征，利用SVM分类器对每个候选区域进行目标类别识别  
  (4)基于提取的深度卷积特征，利用包围盒回归器修正候选区域的位置与尺寸  

  注意：R-CNN 是 2014 年提出的，彼时深度学习在目标检测中刚起步，端到端训练流程尚不成熟，训练检测器还常常结合传统方法，比如：CNN 只被当作特征提取器，分类仍用传统机器学习方法（如 SVM）  
- 训练阶段：  
  - 生成候选区域，归一化到相同分辨率  
  - 利用图像的预先标注，找到与该候选区域重合度最高的真实包围盒，将候选区域赋予真实包围盒同样的分类标签  
  - 利用CNN对这些候选区域进行特征提取  
  - 使用这些深度卷积特征，对每个类别训练一个二分类SVM分类器  
  - 引入包围盒回归器，对候选区域的位置和尺寸进行调整  
- 预测阶段：  
  - 生成候选区域，归一化到相同分辨率
  - 利用训练好的CNN，对候选区域进行提取特征  
  - 利用训练好的SVM，对这些特征进行分类，选取score最高的作为最后分类  
  - 利用训练好的包围盒回归器，微调候选区域的尺寸和位置  
  - 非极大值抑制去除冗余的包围盒  

## 12.5 Fast R-CNN(2015年)  
- R-CNN的弊端：
  (1)对于每张图像中的每个候选区域都需要使用CNN主干网络计算一次特征，导致计算效率低  
  (2)候选区域中间重叠较多，进行特征提取时存在重复计算  
  (3)R-CNN的训练过程包含多个训练过程（主干网络的训练、SVM分类器及包围盒回归器的训练），而SVM分类和坐标回归的结果无法反馈给前端的CNN主干网络用以更新网络参数。  
- Fast R-CNN的重大改进：  
  (1)不再对每个候选区域单独提取特征，而是在**提取整幅图像的特征图**后，直接从特征图提取每个候选区域的特征，避免了多次用主干网络计算特征的过程。  
  (2)Fast R-CNN的候选区域特征提取与分类器及回归器的训练是一个**端到端**的过程，分类和回归的结果可以反馈给主干网络用以更新网络参数。  
- Fast R-CNN的主要步骤：
  (1)首先，图像经过选择性搜索（Selective Search 简称SS）算法提取出约2000个候选区域框，这2000个候选区域框基本上包括了图像中可能出现的目标物体。  
  (2)将图像输入预训练好的CNN网络中，直到最后一层卷积层，在基于原始输入图像中找到约2000个候选框的位置，然后从**图像空间映射到特征图空间**，得到RoI Pooling层。  
  (3)将每一个映射后的RoI划分为固定大小的网格，并且对每个小网格的所有值取得最大值（即Max Pooling池化操作），得到固定大小的特征图。  
  (4)将RoI Pooling层得到的特征图作为后续FC层的输入，最后一层输出**n个分类信息和4个Bounding Box修正偏移量**。将Bounding Box按照位置偏移量进行修正，再根据NMS算法对所有的Bounding Box进行筛选，即可得到对该图像的最终Bounding Box预测值和每个Bounding Box对应的分类概率。  

## 12.6 Faster R-CNN(2015年)  
- Fast R-CNN改善了R-CNN的若干问题，但是仍然采用了Selective  Search的候选区域提取方法，这个步骤也相当耗时。  
- Faster R-CNN提出了**候选区域网络(Region Proposal Network, RPN)**实现了候选区域提取。**Faster R-CNN可以看作RPN+Fast R-CNN**  
- RPN详细工作原理：  
  - 输入：主干网络的卷积层输出的特征图(C,H,W)  
  - 过程：  
  (1)轻量卷积网络滑动窗口滑过特征图上每一个点，生成k个锚框，一共是H\*W\*k个锚框  
  (2)上述锚框分别经过分类层和回归层，得到2k个置信度分数和4k个包围盒的坐标偏移量参数  
  (3)筛选出来的候选区域再经过NMS非极大值抑制，以及一些其他阈值处理，进一步筛选  
  (4)将锚框坐标映射到原图坐标，由此产生最终ROIS
  (5)其中，锚框的真实分类和真实包围盒由若干细节决定，参与loss计算  
  - 输出：n个最终候选区域  
- Faster R-CNN中的两大步骤RPN和Fast R-CNN的分别训练会以不同方式影响主干网络backbone的参数，因此使用了"四步交替法"进行训练。  
  - step1：**训练 RPN（使用 ImageNet 预训练模型）**  
  - step2：**训练 Fast R-CNN 分类器（使用固定 proposal）**  
  - step3：**微调 RPN（共享特征）**  
  - step4: **微调 Fast R-CNN 分类器（共享特征）**  


## 12.7 Faster R-CNN项目  
https://github.com/jiguotong/faster-rcnn  